In [ ]:
import os
import sys
from pathlib import Path as Pt
from typing import Dict, List, Optional, Set, Tuple, Union

import numpy as np

In [ ]:
def custom_strtobool(val: str) -> bool:
    val_lower = val.strip().lower()
    truthy = {"y", "yes", "t", "true", "on", "1"}
    falsey = {"n", "no", "f", "false", "off", "0"}
    if val_lower in truthy:
        return True
    if val_lower in falsey:
        return False
    raise ValueError(f"Недопустимое булево значение: {val}")


def load_cfg(
    path_to_config: str,
    keys_to_bool: set[str] = None,
    keys_to_int: set[str] = None,
    keys_to_float: set[str] = None,
    keys_to_str: set[str] = None,
    com_line: str = "/////////////////////// | Для коментарів | /////////////////////////",
) -> dict[str, object]:
    keys_to_bool = keys_to_bool or {"Px", "Py", "Pz"}
    keys_to_int = keys_to_int or {
        "Seed",
        *["Sx", "Sy", "Sz"],
        "mode",
        *["AddI", "AddFrom", "RemI", "RemFrom"],
        "LoadPrev",
        *["StepLim", "PrintI", "WriteI"],
    }
    keys_to_float = keys_to_float or {
        "T",
        *["Ax", "Ay", "Az"],
        *["g100", "g010", "g001"],
        *["dg", "C_eq", "C0", "N_tot", "N0_cr", "p_b"],
    }
    keys_to_str = keys_to_str or {"DirPrefix"}

    converters: dict[str, callable] = {}
    converters.update({k: int for k in keys_to_int})
    converters.update({k: float for k in keys_to_float})
    converters.update({k: custom_strtobool for k in keys_to_bool})

    text = Pt(path_to_config).read_text(encoding="utf-8")
    lines = [ln.strip() for ln in text.splitlines()]

    try:
        split_idx = lines.index(com_line)
    except ValueError:
        split_idx = len(lines)

    cfg: dict[str, object] = {}
    for raw in lines[:split_idx]:
        if not raw or raw.startswith("#") or raw.startswith("; "):
            continue
        if ":" not in raw:
            raise ValueError(f"Неправильный формат строки: '{raw}'")
        key, val = (part.strip() for part in raw.split(":", 1))

        if key in converters:
            try:
                cfg[key] = converters[key](val)
            except Exception as e:
                raise ValueError(f"Ошибка конвертации ключа '{key}': {e}")
        else:
            if (
                key in keys_to_str
                and len(val) >= 2
                and val[0] == val[-1]
                and val[0] in {"'", '"'}
            ):
                val = val[1:-1]
            cfg[key] = val

    return cfg


In [ ]:
load_cfg(
    path_to_config=r"Rn1\1751191981831466_Rn1_X51Y51Z51_T3e2_dg1.89688e-21\InitSettings.ini"
)

In [ ]:
def process_directory(dir_path: Pt) -> Tuple[Dict, Optional[np.ndarray]]:
    try:
        path_cfg = dir_path / "InitSettings.ini"
        path_history = dir_path / "sim_history.txt"

        if not (path_cfg.exists() and path_history.exists()):
            raise FileNotFoundError(f"Required files missing in {dir_path}")

        data_ini = load_cfg(path_to_config=path_cfg)
        run_id = dir_path.name.split("_")[0]
        data_ini["id"] = run_id

        history_txt = path_history.read_text(encoding="utf-8")
        history_data = np.array(
            [
                *map(
                    lambda line: [*map(float, line.split(":"))],
                    history_txt.splitlines(),
                )
            ]
        )
        # print(history_data)
        (
            n_gas_history,
            n_crystal_history,
            concentration_history,
            delta_gibbs_history,
            energy_change_history,
            crystal_sx_history,
            crystal_sy_history,
            crystal_sz_history,
            mk_step_history,
        ) = history_data

        # print(f"n_gas_history: {n_gas_history}")
        # print(f"n_crystal_history: {n_crystal_history}")
        # print(f"concentration_history: {concentration_history}")
        # print(f"delta_gibbs_history: {delta_gibbs_history}")
        # print(f"energy_change_history: {energy_change_history}")
        # print(f"crystal_sx_history: {crystal_sx_history}")
        # print(f"crystal_sy_history: {crystal_sy_history}")
        # print(f"crystal_sz_history: {crystal_sz_history}")
        # print(f"mk_step_history: {mk_step_history}")

        return delta_gibbs_history[-1], mk_step_history[-1]



    except Exception as e:
        print(f"Error processing directory {dir_path}: {str(e)}")
        raise


base_folder = Pt(
    r"Rn1"
)

res = []

try:
    for root, _, files in os.walk(base_folder):
        if {"InitSettings.ini", "sim_history.txt"}.issubset(files):
            print(f"Processing directory: {root}")
            dg, mk = process_directory(Pt(root))
            res.append([dg, mk])
except Exception as e:
    print(f"Error in main loop: {str(e)}")
    raise

res = np.array(res)
dgs, mks = res.T

In [ ]:
recalculated_write_i = (mks / 102).astype(int) + 1
recalculated_write_i

In [ ]:
mks[15], mks[15] / 102, recalculated_write_i[15], recalculated_write_i[15] * 102

In [ ]:
combs_txt = (base_folder / "WorkCombs.txt").read_text(encoding="utf-8")
combs_lines = combs_txt.splitlines()

In [ ]:
res1 = np.vstack([dgs, recalculated_write_i]).T

combs_lines_new = []

for line_id, line in enumerate(combs_lines):
    items = line.split(" +| ")
    # print(items)

    dg_new, wrti = res1[line_id]

    items[16], items[29] = f"{dg_new:.5e}", f"{int(wrti)}"

    line_new = " +| ".join(items)

    # print(line_new)

    combs_lines_new.append(line_new)

In [ ]:
(base_folder / "WorkCombs_new.txt").write_text(
    "\n".join(combs_lines_new), encoding="utf-8"
)